# <center> HomeWork Assignment - Saving csv to DB
***


<div class="alert alert-block alert-warning">
<h2>Improvements:</h2>
    <ol><li> <b>Dynamic solution:</b> Table and Columns are created dynamically from file</li>
        <li><b>OOPS based:</b> Modular code</li>
        <li><b> Bulk insert:</b> used execute many with batch size, batch size is configurable, (<b><i>default = 5000 rows</i></b>)</li>
    </ol>
</div>

#### File_Processor Class
***

In [18]:
import csv
import mysql.connector as con


class File_Processor:
    def __init__(self,filename,mydb):
        self.filename = filename
        self.mydb = mydb
        if ( not (mydb.is_connected())):
            print( 'DB connctions failed, please check connection attributes')
    # Helper Function to find total rows
    def total_lines(self):
        with open(self.filename) as f:
          return sum(1 for line in f)      
        
    def save_to_DB(self , batchsize= 5000):
        filename = self.filename
        #Call helper function to find rows, exclude header row (-1)
        total_rows_infile = self.total_lines() -1
        rows_remaining = total_rows_infile
        
        with open(self.filename, mode='r') as csv_file:
         csv_reader = csv.reader(csv_file,delimiter=';')
         self.tbl_name = csv_file.name.split('.')[0]
         header_row = next(csv_reader)
         self.setup_Table(header_row)
         row_count = 0
         max_size = batchsize
         print('batch size is {}'.format(max_size) )
         chunk_size = 0
         row_list = []
         for row in csv_reader:
            #if( chunk_size <=max_size):
            row_list.append(tuple(row))
            chunk_size+=1         
            if( (chunk_size >=max_size) or (max_size >= rows_remaining )):
                self.save_row(row_list)
                row_count += chunk_size 
                rows_remaining = total_rows_infile - row_count       
                chunk_size = 0
                row_list.clear()               
         print( '{} Rows inserted in to table {}'.format(row_count,self.tbl_name))
         
    # Function to create table
    
    def setup_Table(self,header_row):
        
        tbl_name = self.tbl_name
        Col_Names = []
        mydb = self.mydb
        for ele in  header_row:
         Col_Names.append( ele.replace("'", '').replace(' ','_'))
        self.ColumnNames = Col_Names
        
        try:
           
            createsqltable = """CREATE TABLE IF NOT EXISTS """ + tbl_name + " (" + " VARCHAR(250),".join(Col_Names) + " VARCHAR(250))"
            cursor = mydb.cursor()
            cursor.execute(createsqltable)
            mydb.commit()
        except Exception as e:
            mydb.close()
            print(str(e))
            
    # Function to insert rows in to Table  
    
    def save_row(self,row):
        
       
        try:
            mydb = self.mydb
            colum_names = self.ColumnNames
            table_name =  self.tbl_name
            query = """insert into """+ table_name + "({0}) values ({1})"
            query = query.format(','.join(colum_names), ','.join('?' * len(colum_names)))
            query1 = query.replace('?','%s')
            cursor = mydb.cursor()
            cursor.executemany(query1, row)
            mydb.commit()
        except Exception as e:
             mydb.close()
             print(str(e))
   
        
    


    

<div class="alert alert-block alert-warning">
<h2>Steps to Execute:</h2>
    <ol><li> <b>Create file processorConnection object :</b><br/><span style ="color:blue" ><i>  mydb = con.connect(host="localhost", user="admin",passwd="admin",use_pure=True,db='ineuron_db')</span>  </li>
        <li> <b>Create file processor object:</b><br/><span style ="color:blue" ><i> pf =  File_Processor('carbon_nanotubes.csv',mydb)</span></li>
        <li><b>call Save_to_DB:</b> <br/> <span style ="color:blue" ><i> pf.save_to_DB(batchsize=10000)</span></li>
    </ol>
</div>

#### Execution
***

In [17]:
# Execution code
        
mydb = con.connect(host="localhost", user="admin",passwd="admin",use_pure=True,db='ineuron_db')       
pf =  File_Processor('carbon_nanotubes.csv',mydb)
pf.save_to_DB( batchsize = 10700)

batch size is 10700
10721 Rows inserted in to table carbon_nanotubes
